In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from abcd.local.paths import core_path, output_path
from abcd.data.read_data import get_subjects_events, add_event_vars

In [3]:
# Fetch subjects and events
subjects_df, events_df = get_subjects_events()

In [4]:
# Filter out events to only leave the baselines
subjects_df, events_df = get_subjects_events()
baseline_events_df = events_df[events_df["eventname"] == "baseline_year_1_arm_1"] 
print("There are {} baseline events for {} subjects!".format(len(baseline_events_df), len(subjects_df)))

There are 9530 baseline events for 9879 subjects!


In [5]:
# Add mental health outcomes
MH_SCORES = {"cbcl_scr_syn_anxdep_r": "Anxious/Dep.",
             "cbcl_scr_syn_withdep_r": "Depression",
             "cbcl_scr_syn_somatic_r": "Somatic",
             "cbcl_scr_syn_social_r": "Social",
             "cbcl_scr_syn_attention_r": "Attention",
             "cbcl_scr_syn_rulebreak_r": "Rule-breaking",
             "cbcl_scr_syn_aggressive_r": "Aggressive",
             "cbcl_scr_syn_internal_r": "Internalizing",
             "cbcl_scr_syn_external_r": "Externalizing"}    

mental_health_file = os.path.join(core_path, "mental-health", "mh_p_cbcl.csv")
mh_events_df = add_event_vars(baseline_events_df, mental_health_file, vars=list(MH_SCORES.keys()))
print("From those, there are {} events with mental health scores".format(len(mh_events_df)))

From those, there are 9528 events with mental health scores


c:\users\camgonza\desktop\abcd5\abcd\utils\io.py:24: DtypeWarning: Columns (124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=sep)


In [6]:
# Count missing values
"""
for column in mh_events_df.columns:
    nr_missing_values = mh_events_df[[column]].isna().sum().sum()
    if nr_missing_values > 0:
        print("Column {} has {} missing values".format(column, nr_missing_values))
"""
# Remove missing values
mh_events_df = mh_events_df.dropna()
print("After removing missing values, the df has {} events".format(len(mh_events_df)))

After removing missing values, the df has 9495 events


In [7]:
# Plot distribution of variables
import pygal
from pygal import Config
from abcd.plotting.pygal.rendering import display_html

plot = pygal.Box()
plot.title = 'Child behavior checklist CBCL syndrome scales'
for var_id, var_name in MH_SCORES.items():
    plot.add(var_name, list(mh_events_df[var_id]))
display_html(plot)


In [17]:
from abcd.data.define_splits import save_restore_sex_fmri_splits
from abcd.data.NETWORKS import CONNECTIONS
from sklearn.linear_model import LinearRegression, SGDRegressor
from abcd.analysis.methods.sklearn_fitting import calculate_regession_results, plot_results_one_site

site_splits = save_restore_sex_fmri_splits(k=5)
feature_columns = CONNECTIONS

In [18]:
# Linear Regression results
y_column = "cbcl_scr_syn_internal_r"
model = LinearRegression()
results = calculate_regession_results(model, mh_events_df, site_splits, feature_columns, y_column)
results      

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:03<00:00,  5.90it/s]


,split,MAE mean,MAE std,Max. error mean,Max. error std
0,Train,3.952788,0.041735,42.826578,2.158057
1,ID Test,3.967567,0.057417,36.947226,1.351567
2,OOD Test,3.938181,0.266546,28.703506,5.129957


In [16]:
plot_results_one_site(model, mh_events_df, site_splits, feature_columns, y_column, y_column_name=MH_SCORES[y_column], site_id="site01")

In [ ]:
#